In [1]:
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

NFLX = '/kaggle/input/netflix-stock-price-data-for-2023-24/NFLX.csv'
# there have been no adjustments in the period of interest so we can drop the adjusted close data
df = pd.read_csv(filepath_or_buffer=NFLX, parse_dates=['Date']).drop(columns=['Adj Close'])
df['month'] = df['Date'].dt.month
df['high/low'] = df['High'] - df['Low']
df.head()

,Date,Open,High,Low,Close,Volume,month,high/low
0,2023-02-01,353.859985,365.390015,349.910004,361.989990,8005200,2,15.480011
1,2023-02-02,365.160004,368.320007,358.429993,366.890015,7857000,2,9.890014
2,2023-02-03,359.079987,379.429993,359.000000,365.899994,9402000,2,20.429993
3,2023-02-06,363.640015,368.450012,360.679993,361.480011,4994900,2,7.770019
4,2023-02-07,358.510010,364.179993,354.179993,362.950012,6289400,2,10.000000


In [2]:
from plotly.express import line
prices = ['Open', 'High', 'Low', 'Close', ]
line(data_frame=df, x='Date', y=prices)

In [3]:
from plotly.express import scatter
scatter(data_frame=df, x='Date', y=prices, trendline='ols', trendline_scope='overall').show()
scatter(data_frame=df, x='Date', y=prices, trendline='lowess', trendline_scope='overall').show()

It has been a volatile period for Netflix.

Let's look at the 50-day and 200-day moving averages.

In [4]:
close_df = df[['Date', 'Close']].copy()
close_df['50d'] = close_df['Close'].rolling(window=50, ).mean()
# because we don't have much data we're going to bootstrap the 200d average
close_df['200d'] = close_df['Close'].rolling(window=200, min_periods=50).mean()
close_df = close_df.dropna() 
line(data_frame=close_df, x='Date', y=['Close', '50d', '200d']).show()

Our simple algorithmic trading 'algorithm' is almost indistinguishable from buy and hold; we buy during the 200d bootstrapping period and hold to the end, even though we participate in the entire decline from mid-July to mid-October and briefly actually lose money.

In [5]:
scatter(data_frame=df, x='Close', y='Volume', color='month', hover_data=['Date'], log_y=True)

Price and volume are not especially correlated; almost every month and every price level has high volume days.

In [6]:
scatter(data_frame=df, x='high/low', y='Volume', color='month', hover_data=['Date'], log_y=True, trendline='ols')

Although, and this is kind of interesting, the volume does seem to be a function of the difference between the high and low price, meaning that volume is driven by price movements (up or down), but the R2 is only .3.